In [ ]:
import logging
logging.basicConfig(level=logging.ERROR)  # Only show ERROR level and above
import pandas as pd
import os
import sys
import seaborn as sns
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
from src.cmos_plotter.Plotter_Helper_KV import *
from src.utils.metadata_functions import load_metadata_as_dataframe
from src.cmos_plotter import Conduction_speed_plotter as csp
import pickle
import matplotlib.pyplot as plt


In [ ]:
MAIN_PATH = '...'
CONDUCTION_SPEEDS_PATH = '...'
with open(os.path.join(MAIN_PATH, '.../validated_results_full_data_w_speed_and_firing_1.pkl'), 'rb') as f:
    df_1 = pd.DataFrame(pickle.load(f))
with open(os.path.join(MAIN_PATH, '.../validated_results_full_data_w_speed_and_firing_2.pkl'), 'rb') as f:
    df_2 = pd.DataFrame(pickle.load(f))


In [ ]:
def custom_lineplot(data, x_metric, y_metric, axes, label, fontsize=16, labelsize=14, xlim = None, ylim = None, save_path=None):
    plt.rcParams['xtick.bottom'] = True    
    plt.rcParams['ytick.left'] = True    
    plt.rcParams['font.family'] = 'Arial'    
    sns.lineplot(data=data, x=x_metric, y=y_metric, color="dimgrey", markers=True, estimator="mean", 
                 err_style="bars", legend=True, zorder=1, ax=axes, dashes=False, marker="o", ms=10, linewidth=0.5, errorbar=("ci", 95))        
    axes.set_ylabel(label, fontsize=fontsize)    
    axes.set_xlabel("Day In Vitro", fontsize=fontsize)    
    axes.grid(True, linestyle=':', alpha=0.7)    
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    if xlim is not None:
        axes.set_xlim(xlim)
    if ylim is not None:
        axes.set_ylim(ylim)
    if save_path:
        plt.savefig(os.path.join(save_path, f'{label[:-3]}.png'), dpi=300)
        plt.savefig(os.path.join(save_path, f'{label[:-3]}.pdf'), dpi=300)
    

In [ ]:
def multi_dataframe_lineplot(data_list, x_metrics, y_metrics, axes, labels, colors, fontsize=16, labelsize=14, xlim=None, ylim=None, save_path=None, log_scale=False):
    plt.rcParams['xtick.bottom'] = True    
    plt.rcParams['ytick.left'] = True    
    plt.rcParams['font.family'] = 'Arial'
    
    # Plot each dataframe with its own color
    for data, x_metric, y_metric, label, color in zip(data_list, x_metrics, y_metrics, labels, colors):
        sns.lineplot(data=data, x=x_metric, y=y_metric, color=color, markers=True, estimator="mean", 
                    err_style="bars", label=label, zorder=1, ax=axes, dashes=False, 
                    marker="o", ms=10, linewidth=0.5, errorbar=("ci", 95))
    
    # Set common axes properties
    axes.set_ylabel(labels[0], fontsize=fontsize)  # Using the first label for y-axis title
    axes.set_xlabel("Log (Day In Vitro)", fontsize=fontsize)    
    axes.grid(True, linestyle=':', alpha=0.7)    
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    axes.legend()
    
    if log_scale:
        axes.set_xscale('log')  # Set x-axis to log scale
    

    if xlim is not None:
        axes.set_xlim(xlim)
    if ylim is not None:
        axes.set_ylim(ylim)
    
    if save_path:
        plt.savefig(os.path.join(save_path, f'conduction_speeds_combined.png'), dpi=300)
        plt.savefig(os.path.join(save_path, f'conduction_speeds_combined.pdf'), dpi=300)

In [ ]:
# Filter data if needed (like in your example with speeds)
df_1 = pd.DataFrame(df_1)
df_2 = pd.DataFrame(df_2)
df_2 = df_2[df_2['div'] > 10]
df_1 = df_1[df_1['div'].isin([23, 65, 107, 129, 147])]
print(df_1.shape, df_2.shape)

# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 6))

# Define colors - dimgray for first and a lighter gray for second
colors = ['dimgray', '#A9A9A9']  # dimgray and darkgray

# Call the modified function with both dataframes
multi_dataframe_lineplot(
    data_list=[df_1, df_2],
    x_metrics=[df_1['div'], df_2['div']],
    y_metrics=[df_1['speed_ms-1'], df_2['speed_ms-1']],
    axes=ax,
    labels=['Conduction Speed [m/s] - Group 1', 'Conduction Speed [m/s] - Group 2'],
    colors=colors,
    fontsize=16, 
    labelsize=14, 
    xlim=None, 
    ylim=(0.35, 0.8), 
    save_path=CONDUCTION_SPEEDS_PATH,
    log_scale=True
)

plt.tight_layout()
plt.show()

In [ ]:
MAIN_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/1_Subprojects/Neurons_As_DNNs/3_Processed_Data/Pickle_files_1851'

#RECORDINGS_PATH = os.path.join(MAIN_PATH,f'Raw_Traces')
RECORDINGS_PATH = '/itet-stor/kvulic/neuronies/single_neurons/1_Subprojects/Small_Network_Plasticity/2_Raw_Data/Plasticity_Experiments/Raw_Data/BCM_Test/1821'
SORTER_PATH = os.path.join(MAIN_PATH, f'Sorters_old_version')
CONDUCTION_SPEEDS_PATH = os.path.join(MAIN_PATH, f'Results/Conduction_speeds')

if not os.path.exists(CONDUCTION_SPEEDS_PATH):
    os.makedirs(CONDUCTION_SPEEDS_PATH)

metadata = load_metadata_as_dataframe(file_path=MAIN_PATH)


In [ ]:
# Example usage:
df = csp.create_summary_visualization(CONDUCTION_SPEEDS_PATH, CONDUCTION_SPEEDS_PATH, group_by='div')

